In [1]:
import andes
from andes.interop.pandapower import to_pandapower, make_GSF
import pandapower as pp

from numpy import pi
from andes.shared import pd, rad2deg, deg2rad

import numpy as np
import andes.interop.pandapower as adpp

import pandas as pd

import andes
import numpy as np
from andes.interop.pandapower import to_pandapower, make_link_table, runopp_map
from andes.interop.pandapower import add_gencost, build_group_table
andes.config_logger(stream_level=20)

import pandas as pd
import pandapower as pp
from math import ceil, floor

import os

import matplotlib.pyplot as plt

import scipy.stats as stat

from jams import rted2, dcopf, rted3

In [2]:
# !andes run '/Users/jinningwang/Documents/work/andes/jwang/case/case2000.m' --convert

In [3]:
print(andes.__version__)
print(pp.__version__)

1.7.1.post214.dev0+gb93eafa2
2.8.0


In [4]:
# ss0 is used for PP conversion

case2k = '/Users/jinningwang/Documents/work/andes/jwang/case/case2000.xlsx'
case14 = andes.get_case('ieee14/ieee14_ac8b.xlsx')
wecc = andes.get_case('wecc/wecc_full.xlsx')
npcc = andes.get_case('npcc/npcc.xlsx')
ssa = andes.load(case2k,
                 setup=True,
                 no_output=True,
                 default_config=False)

Working directory: "/Users/jinningwang/Documents/work/andes/jwang/notes2"
> Loaded config from file "/Users/jinningwang/.andes/andes.rc"
> Loaded generated Python code in "/Users/jinningwang/.andes/pycode".
Generated code for <PQ> is stale.
Numerical code generation (rapid incremental mode) started...


Generating code for 1 models on 8 processes.


Saved generated pycode to "/Users/jinningwang/.andes/pycode"
> Reloaded generated Python code of module "pycode".
Generated numerical code for 1 models in 0.3712 seconds.
Parsing input file "/Users/jinningwang/Documents/work/andes/jwang/case/case2000.xlsx"...
Input file parsed in 1.5020 seconds.
System internal structure set up in 0.0981 seconds.


In [5]:
# --- conversion ---
ssp = to_pandapower(ssa)

# add gen cost, G1-11. G10 EV, G11 Slack
linearcost = [1] * ssp.gen.shape[0]

gen_cost = np.array([[2., 0., 0., 3., 0., 0., 0.]] * ssp.gen.shape[0])
gen_cost[:, 5] = linearcost  # c1

add_gencost(ssp, gen_cost)

-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0082 seconds.
0: |F(x)| = 35.37894339
1: |F(x)| = 0.2692108655
2: |F(x)| = 0.0002885472789
3: |F(x)| = 9.353549046e-10
Converged in 4 iterations in 0.0604 seconds.
Power flow results are consistent. Conversion is successful.


True

In [6]:
pp.rundcopp(ssp)

In [8]:
ssd = dcopf()
ssd.from_andes(ssa)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-04-08


-> System connectivity check results:
  No islanded bus detected.
  System is interconnected.
  Each island has a slack bus correctly defined and enabled.

-> Power flow calculation
           Numba: Off
   Sparse solver: KLU
 Solution method: NR method
Power flow initialized in 0.0092 seconds.
0: |F(x)| = 35.37894339
1: |F(x)| = 0.2692108655
2: |F(x)| = 0.0002885472789
3: |F(x)| = 9.353549046e-10
Converged in 4 iterations in 0.0514 seconds.
Power flow results are consistent. Conversion is successful.
GSF is consistent.
/Users/jinningwang/Documents/work/mambaforge/envs/andes/lib/python3.9/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [9]:
ssd.get_res()

Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


Successfully build DCOPF model.


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6413 rows, 544 columns and 82808 nonzeros
Model fingerprint: 0xa53fbf8f
Coefficient statistics:
  Matrix range     [1e-07, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [5e-03, 1e+01]
  RHS range        [2e+01, 4e+03]

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 6413 rows and 544 columns
Presolve time: 0.01s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.7109210e+02   0.000000e+00   0.000000e+00      0s

Solved with dual simplex
Solved in 0 iterations and 0.02 seconds (0.01 work units)
Optimal objective  6.710921000e+02


Successfully solve DCOPF.


,gen,pg
0,1,1.5825
1,2,0.2573
2,3,0.6187
3,4,0.1000
4,5,1.4963
...,...,...
539,541,0.0000
540,542,1.7460
541,543,0.0000
542,544,0.0000
